In [ ]:
# This script is designed to do somee simple tests for Enterprise Search using Elastic Cloud
# It will use the Elastic Cloud CLI. Please asetup as per the docs
# https://www.elastic.co/guide/en/ecctl/current/index.html
# on Mac use
# brew tap elastic/tap
# brew install elastic/tap/ecctl
# ecctl init 
# Need to add the App Search specific python client
# python -m pip install elastic-app-search


In [ ]:

engine_name = "sizing"
cluster_info_path= "./cluster-info.json"


In [ ]:
import os
import json
import time
import requests
import subprocess
from requests.auth import HTTPBasicAuth
from urllib.parse import urljoin
from elastic_app_search import Client

In [ ]:
# So there is currently a bug in ecctl which should be fixed soon.
# until then please run the command directly in the terminal to create the file.
# when the issue is fixed we can reuse this code
#
# ecctl deployment create -f ./deployment-config.json > cluster-info.json

#print("Checking if the cluster is already created")
#if not(os.path.exists(cluster_info_path) and os.path.getsize(cluster_info_path)>0):
#    print("No cluster info file so lets create a new cluster in cloud")
#    info_file=open(cluster_info_path,"w+")
#    out = subprocess.run(args=['ecctl', 'deployment', 'create', '-f ./deployment-config.json'], 
#           stdout=info_file, 
#           stderr=subprocess.STDOUT)
#    stdout,stderr = out.communicate()
#    print(stdout,stderr)
        

In [ ]:

print("Reading cluster info file: ",cluster_info_path)
with open(cluster_info_path) as json_file:
    cluster_info = json.load(json_file)
    deployment_name = cluster_info['name']
    #print("Cluster name: ",deployment_name)
    deployment_id = cluster_info['id']
    #print("Cluster ID: ",deployment_id)
    cloud_id = cluster_info['resources'][0]['cloud_id']
    #print("CloudID: ",cloud_id)
    elastic_password = cluster_info['resources'][0]['credentials']['password']
    #print("elastiic password: ",elastic_password)
    cluster_exists = True

In [ ]:
# To get the endpoints for App Search, Elasticsearch etc we need to run a second command
# This will get all the details of the cluster and return json
out = subprocess.run(['ecctl', 'deployment', 'show', deployment_id], 
           stdout=subprocess.PIPE, 
           stderr=subprocess.STDOUT)
cluster_details = json.loads(out.stdout)

In [ ]:
# Now we can extract any details we need
as_endpoint = cluster_details['resources']['enterprise_search'][0]['info']['metadata']['endpoint']
#print('App Search endpoint: ',as_endpoint)
es_endpoint = cluster_details['resources']['elasticsearch'][0]['info']['metadata']['endpoint']
#print('Elasticsearch endpoint: ',es_endpoint)



In [ ]:
# The App Search Client needs an API key so we will need to call the API directly using basic auth to generate a key

# The only way to get this atm is through an undocumented API
create_key_url = urljoin("https://"+as_endpoint, '/as/credentials/collection?page[current]=1')
#print(create_key_url)
response =requests.get(create_key_url, auth=('elastic', elastic_password))
#print(response)
api_key_info = response.json()
for key in api_key_info['results']:
    if key['type'] == "private":
        private_key = key['key']
        #print("Private key: ",private_key)
    elif key['type'] == "search":
        search_key = key['key']
        #print("Search key: ",search_key)
#as_client = Client(

In [ ]:
# The App Search client need s base endpoint which included the API path 
base_endpoint = as_endpoint+'/api/as/v1'
as_client = Client(
    base_endpoint=base_endpoint,
    api_key=private_key
)

In [ ]:
as_client.create_engine(engine_name, 'en')

In [ ]:
# Lets do a simple ingest and query before we delete the enging

documents = [
    {
      'id': 'INscMGmhmX4',
      'url': 'https://www.youtube.com/watch?v=INscMGmhmX4',
      'title': 'The Original Grumpy Cat',
      'body': 'A wonderful video of a magnificent cat.'
    },
    {
      'id': 'JNDFojsd02',
      'url': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ',
      'title': 'Another Grumpy Cat',
      'body': 'A great video of another cool cat.'
    }
]

as_client.index_documents(engine_name, documents)

In [ ]:
# Lets search for something
as_client.search(engine_name, 'grumpy cat', {})

In [ ]:
# Now let's destroy the engine
